# Initial setting

## libraries

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css"
).text
HTML(styles)

In [2]:
import re
import requests
from time import sleep
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

## dataset

In [3]:
# Load data
fifa15 = pd.read_csv('data/players_15.csv')
fifa16 = pd.read_csv('data/players_16.csv')
fifa17 = pd.read_csv('data/players_17.csv')
fifa18 = pd.read_csv('data/players_18.csv')
fifa19 = pd.read_csv('data/players_19.csv')
fifa20 = pd.read_csv('data/players_20.csv')
team_league = pd.read_csv('data/teams_and_leagues.csv')


In [4]:
fifa20.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,...,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,2004-07-01,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,"Beat Offside Trap, Argues with Officials, Earl...",88,95,70,92,88,97,...,91,84,93,95,95,86,68,75,68,94,48,40,94,94,75,96,33,37,26,6,11,15,14,8,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,92+2,87+2,87+2,87+2,92+2,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2018-07-10,2022.0,LS,7.0,90.0,93.0,82.0,89.0,35.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,"Long Throw-in, Selfish, Argues with Officials,...",84,94,89,83,87,89,...,89,91,87,96,71,95,95,85,78,93,63,29,95,82,85,95,28,32,24,7,11,15,14,11,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",CAM,10.0,NaN,2017-08-03,2022.0,LW,10.0,91.0,85.0,87.0,95.0,32.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,"Power Free-Kick, Injury Free, Selfish, Early C...",87,87,62,87,87,96,...,94,89,96,92,84,80,61,81,49,84,51,36,87,90,90,94,27,26,29,9,9,15,15,11,84+3,84+3,84+3,90+3,89+3,89+3,89+3,90+3,90+3,90+3,90+3,89+3,82+3,82+3,82+3,89+3,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right,3,3,1,Medium/Medium,Normal,Yes,164700000.0,NaN,GK,13.0,NaN,2014-07-16,2023.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,92.0,78.0,89.0,52.0,90.0,"Flair, Acrobatic Clearance",13,11,15,43,13,12,...,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,27,12,18,87,92,78,90,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",LW,7.0,NaN,2019-07-01,2024.0,LF,10.0,91.0,83.0,86.0,94.0,35.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,"Beat Offside Trap, Selfish

To do  
1. create year variable DONE
2. create team name variable DONE
3. merge all years DONE
4. create dummies for player_tags DONE
5. create dummies for player_traits DONE
6. create dummies for team_position and select primary position for each player DONE
7. remove +- signs from abilities DONE
8. drop unnecessary columns DONE

# Data cleaning

## team_league.csv - You can skip this section as it takes forever

In [5]:
## set a base url for team pages
#base_url = 'https://sofifa.com/team/'
#
## create an empty list to store team names
#team_name = []
#
## for each team url
#for team in team_league.url:
#    
#    # open team page and get a page title
#    team_url = base_url + str(team)
#    soup = BeautifulSoup(requests.get(team_url).text, 'html.parser')
#    title = soup.find_all('title')
#    
#    # delete redundant strings and append it to the list
#    name = re.sub(' FIFA.*', '', title[0].contents[0])
#    team_name.append(name)

In [6]:
## concat the original team_league and the team_name list
#df_team = pd.concat([team_league, pd.Series(team_name)], axis=1)
#df_team.columns = ['url', 'league_name', 'club']
#df_team.league_name = df_team.league_name.str.strip()
#
## save a csv file
#df_team.to_csv('data/df_team.csv', index=False)

## player datasets - You can resume from here

### merge all years

In [7]:
# read df_team
df_team = pd.read_csv('data/df_team.csv')
df_team = df_team[['club','league_name']].drop_duplicates()

# add year variable for each player dataset
fifa15['year'] = 2015
fifa16['year'] = 2016
fifa17['year'] = 2017
fifa18['year'] = 2018
fifa19['year'] = 2019
fifa20['year'] = 2020

# merge all datasets and add league name
fifa = pd.concat([fifa15, fifa16, fifa17, fifa18, fifa19, fifa20])
df_fifa = pd.merge(fifa, df_team, on='club', how='left')

### player traits and tags

In [9]:
# create dummy variables for player traits
# first make a list for all player traits (without CPU AI Only)
all_traits = []
player_traits = [x for x in df_fifa.player_traits.unique() if str(x) != 'nan']

for traits in player_traits:
    #traits_list =traits.split(',')
    traits_list = [x for x in traits.split(',') if 'CPU AI Only' not in x]
    for trait in traits_list:
        all_traits.append(trait.strip())     

# create a unique set for player traits
all_traits = list(set(all_traits))

# then add dummy variables using this list
for trait in all_traits:
    df_fifa['d_trait_' + trait] = [1 if trait in str(x) else 0 for x in df_fifa.player_traits]

In [10]:
# first make a list for all player tags
all_tags = []
player_tags = [x for x in df_fifa.player_tags.unique() if str(x) != 'nan']

for tags in player_tags:
    tags_list = tags.split(',')
    for tag in tags_list:
        all_tags.append(tag.strip())         

# create a unique set for player tags
all_tags = [x.replace('#','') for x in set(all_tags)]

# then add dummy variables using this list
for tag in all_tags:
    df_fifa['d_tag_' + tag] = [1 if tag in str(x) else 0 for x in df_fifa.player_tags]

### position

In [11]:
# set the first position in player_positions as his main position
df_fifa['main_position'] = [x.split(',')[0] for x in df_fifa['player_positions']]

# create multiple position dummies
df_fifa['d_multiple_position'] = [1 if len(x.split(',')) > 1 else 0 for x in df_fifa['player_positions']]

# make a list for all player positions
all_positions = []
player_positions = [x for x in df_fifa.player_positions.unique() if str(x) != 'nan']

for positions in player_positions:
    positions_list = positions.split(',')
    for position in positions_list:
        all_positions.append(position.strip())         

# create a unique set for player tags
all_positions = list(set(all_positions))

# then add dummy variables using this list
for position in all_positions:
    df_fifa['d_pos_' + position] = [1 if position in str(x) else 0 for x in df_fifa.player_positions]

### variables regarding abilities

In [12]:
# create a list for varibales regarding player abilities
var_ab_list = ['pace','shooting','passing','dribbling','defending','physic']
var_ab_detail = [x for x in df_fifa.columns if x.startswith('gk_') | x.startswith('attacking_') | x.startswith('skill_') | 
                x.startswith('movement_') | x.startswith('power_') | x.startswith('mentality_') | x.startswith('defending_') |
                x.startswith('goalkeeping_')]
var_ab_list = var_ab_list + var_ab_detail

# for each variable
for var_ab in var_ab_list:
    
    # remove '+-' just to focus on basic ability level
    # then remove 'nan' string and convert the series to numeric
    # rename the variable names so that it gets easier to filter variables later
    df_fifa[var_ab] = [re.sub('[+-].*', '', str(x)) for x in df_fifa[var_ab]]
    df_fifa[var_ab] = [x.replace('nan', '') for x in df_fifa[var_ab]]
    df_fifa['ab_' + var_ab] = pd.to_numeric(df_fifa[var_ab])

### miscellaneous

In [18]:
# create a dummy for preferred foot
df_fifa['d_foot_left'] = [1 if x == 'Left' else 0 for x in df_fifa['preferred_foot']]

# create a dummy for European Division 1 leagues
div1_leagues = ['Spain Primera Division','German 1. Bundesliga','French Ligue 1','English Premier League','Italian Serie A']
df_fifa['d_div1_league'] = [1 if x in div1_leagues else 0 for x in df_fifa['league_name']]

# replace whitespace within variable names with underscore
df_fifa.columns = [x.replace(' ', '_') for x in df_fifa.columns]

# select variables
var_list = ['sofifa_id','short_name','age','dob','height_cm','weight_kg','nationality','club','overall','value_eur','wage_eur',
            'preferred_foot','international_reputation','weak_foot','skill_moves','work_rate','body_type','release_clause_eur','year',
            'team_position','team_jersey_number','loaned_from','joined','contract_valid_until','league_name','main_position']
var_others = [x for x in df_fifa.columns if x.startswith('d_') | x.startswith('ab_')]
var_list = var_list + var_others
df_fifa = df_fifa[df_fifa.columns.intersection(var_list)]

### save the final dataset as csv

In [19]:
df_fifa.to_csv('data/df_fifa.csv', index=False)

In [20]:
df_fifa.head()

,sofifa_id,short_name,age,dob,height_cm,weight_kg,nationality,club,overall,value_eur,wage_eur,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,release_clause_eur,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,year,league_name,d_trait_Cautious_With_Crosses,d_trait_Injury_Prone,d_trait_Avoids_Using_Weaker_Foot,d_trait_Backs_Into_Player,d_trait_Flair,d_trait_Saves_with_Feet,d_trait_Leadership,d_trait_Inflexible,d_trait_Finesse_Shot,d_trait_Team_Player,d_trait_1-on-1_Rush,d_trait_Set_Play_Specialist,d_trait_Rushes_Out_Of_Goal,d_trait_Beat_Offside_Trap,d_trait_Early_Crosser,d_trait_Second_Wind,d_trait_Through_Ball,d_trait_Long_Throw-in,d_trait_Giant_Throw-in,d_trait_Diver,d_trait_Acrobatic_Clearance,d_trait_Selfish,d_trait_GK_Up_for_Corners,d_trait_Argues_with_Officials,d_trait_Takes_Finesse_Free_Kicks,...,d_pos_GK,ab_pace,ab_shooting,ab_passing,ab_dribbling,ab_defending,ab_physic,ab_skill_moves,ab_gk_diving,ab_gk_handling,ab_gk_kicking,ab_gk_reflexes,ab_gk_speed,ab_gk_positioning,ab_attacking_crossing,ab_attacking_finishing,ab_attacking_heading_accuracy,ab_attacking_short_passing,ab_attacking_volleys,ab_skill_dribbling,ab_skill_curve,ab_skill_fk_accuracy,ab_skill_long_passing,ab_skill_ball_control,ab_movement_acceleration,ab_movement_sprint_speed,ab_movement_agility,ab_movement_reactions,ab_movement_balance,ab_power_shot_power,ab_power_jumping,ab_power_stamina,ab_power_strength,ab_power_long_shots,ab_mentality_aggression,ab_mentality_interceptions,ab_mentality_positioning,ab_mentality_vision,ab_mentality_penalties,ab_mentality_composure,ab_defending_marking,ab_defending_standing_tackle,ab_defending_sliding_tackle,ab_goalkeeping_diving,ab_goalkeeping_handling,ab_goalkeeping_kicking,ab_goalkeeping_positioning,ab_goalkeeping_reflexes,d_foot_left,d_div1_league
0,158023,L. Messi,27,1987-06-24,169,67,Argentina,FC Barcelona,93,0,0,Left,5,3,4,Medium/Low,Normal,NaN,CF,10.0,NaN,2004-07-01,2018.0,2015,Spain Primera Division,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,93.0,89.0,86.0,96.0,27.0,63.0,4,NaN,NaN,NaN,NaN,NaN,NaN,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,1,1
1,20801,Cristiano Ronaldo,29,1985-02-05,185,80,Portugal,Real Madrid,92,0,0,Right,5,4,5,High/Low,Normal,NaN,LW,7.0,NaN,2009-07-01,2018.0,2015,Spain Primera Division,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,93.0,93.0,81.0,91.0,32.0,79.0,5,NaN,NaN,NaN,NaN,NaN,NaN,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,0,1
2,9014,A. Robben,30,1984-01-23,180,80,Netherlands,FC Bayern München,90,0,0,Left,5,2,4,High/Low,Normal,NaN,SUB,10.0,NaN,2009-08-28,2017.0,2015,German 1. Bundesliga,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,93.0,86.0,83.0,92.0,32.0,64.0,4,NaN,NaN,NaN,NaN,NaN,NaN,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,1,1
3,41236,Z. Ibrahimović,32,1981-10-03,195,95,Sweden,Paris Saint-Germain,90,0,0,Right,5,4,4,Medium/Low,Normal,NaN,ST,10.0,NaN,2012-07-01,2016.0,2015,French Ligue 1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,76.0,91.0,81.0,86.0,34.0,86.0,4,NaN,NaN,NaN,NaN,NaN,NaN,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,NaN,25,41,27,13,15,10,9,12,0,1
4,167495,M. Neuer,28,1986-03-27,193,92,Germany,FC Bayern München,90,0,0,Right,5,4,1,Medium/Medium,Normal,NaN,GK,1.0,NaN,2011-07-01,2019.0,2015,German 1. Bundesliga,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,1,87.0,85.0,92.0,86.0,60.0,90.0,25,25,25,42,25,25,25,25,41,31,58,61,43,89,35,42,78,44,83,25,29,30,25,20,37,NaN,25,25,25,87,85,92,90,86,0,1


In [21]:
df_fifa.shape

(100995, 146)